# Imports

In [61]:
import os
import pandas as pd


from dotenv import load_dotenv
import requests
from sqlalchemy import create_engine

from packages.utils import extract_player_salaries,load_into_db

In [62]:

dotenv_path = ("/Users/maukanmir/Documents/Machine-Learning/Web-Scraping-Code/Player-Salaries/dot.env")
load_dotenv(dotenv_path)

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
TABLE_NAME_PlAYER_SALARIES = "player_salaries"
TABLE_NAME_MASTER_PLAYER_SALARIES = "master_player_salaries"

engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')

# Grabbing Player Salaries from 2011-2024

In [20]:
years = [str(year) for year in range(2011, 2025)]
all_data = pd.DataFrame()

for year in years:
  url = f"https://www.spotrac.com/nba/rankings/player/_/year/{year}/sort/cash_total"
  response = requests.get(url)
  year = int(year)
  df = extract_player_salaries(response.text, year)
  all_data = pd.concat([all_data, df])

# Download Dataset into DB

In [21]:
load_into_db(all_data, engine, TABLE_NAME_PlAYER_SALARIES)

Data successfully written to the database.


# Grabbing Player Salaries From 1989-2024

In [65]:
all_data = pd.DataFrame()
years = [str(year) + "-" + str(year+1) for year in range(1989, 2025)]
for year in years:
  base_url = "https://hoopshype.com/salaries/players/{year}/"
  response = requests.get(base_url)
  df = grab_all_salaries(response.text, year)
  all_data = pd.concat([all_data, df])


In [64]:
def grab_all_salaries(html_content, year:str):
  """_summary_

  Args:
      response (_type_): HTML content
      year (str): _description_

  Returns:
      _type_: _description_
  """

  soup = BeautifulSoup(html_content, "html.parser")
  tds = soup.findAll("td")

  mega_string = " ".join([td.text.strip() for td in tds][1:])
  cleaned_list = [block.strip().split(" ") for block in mega_string.split(".")[1:]]
  df = []
  for block in cleaned_list:
    players = {}
    for idx, player in enumerate(block):
      if "$" in player and "$" in block[idx-1]:
        continue
      elif "$" in player and block[idx-1].isalpha():
        players["salary"] = int(player.replace(",", "").replace("$", ""))
      elif idx ==2 and player.isalpha():
        players["name"] = block[idx-2]+ " " +  block[idx-1] + " " + player
      elif player.isalpha() and block[idx-1].isalpha() and "$" in block[idx+1]:
        players["name"] = block[idx-1] + " " + player
        df.append(players)

  df = pd.DataFrame(df)
  df["season"] = year
  return df

In [67]:
load_into_db(all_data, engine, TABLE_NAME_MASTER_PLAYER_SALARIES)

Data successfully written to the database.
